In [42]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)
data = pd.read_excel('nba_player_data.xlsx')


In [43]:
data.sample(10)

,Year,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
4062,2018-19,Playoffs,203468,6,CJ McCollum,1610612757,POR,16,635,154,350,0.440,46,117,0.393,41,56,0.732,14,66,80,59,13,10,30,34,395,316,1.97,0.43
6842,2022-23,Regular%20Season,203939,227,Dwight Powell,1610612742,DAL,76,1458,197,269,0.732,0,5,0.000,112,168,0.667,149,165,314,69,48,26,65,209,506,770,1.06,0.74
359,2013-14,Regular%20Season,2045,360,Hedo Turkoglu,1610612746,LAC,38,392,42,109,0.385,22,50,0.440,9,18,0.500,11,78,89,33,19,10,16,50,115,174,2.06,1.19
193,2013-14,Regular%20Season,202708,194,Norris Cole,1610612748,MIA,82,2014,207,500,0.414,60,174,0.345,53,68,0.779,17,143,160,248,77,5,127,156,527,582,1.95,0.61
3082,2017-18,Regular%20Season,1628395,306,Jordan Bell,1610612744,GSW,57,809,116,185,0.627,0,4,0.000,30,44,0.682,60,147,207,102,35,56,51,92,262,528,2.00,0.69
1946,2015-16,Playoffs,202334,86,Ed Davis,1610612757,POR,11,205,21,40,0.525,0,2,0.000,19,33,0.576,26,49,75,14,2,7,10,33,61,116,1.40,0.20
1735,2015-16,Regular%20Season,1626202,351,Joe Young,1610612754,IND,41,384,62,169,0.367,10,46,0.217,20,25,0.800,6,44,50,65,15,0,33,30,154,139,1.97,0.46
2689,2016-17,Playoffs,203477,128,Isaiah Canaan,1610612741,CHI,3,95,13,26,0.500,5,14,0.357,4,6,0.667,0,4,4,4,3,0,5,6,35,26,0.80,0.60
4441,2019-20,Regular%20Season,201588,173,George Hill,1610612749,MIL,59,1271,194,376,0.516,81,176,0.460,85,101,0.842,50,127,177,180,47,5,57,84,554,708,3.16,0.82
4524,2019-20,Regular%20Season,1626192,255,Pat Connaughton,1610612749,MIL,67,1243,137,301,0.455,55,166,0.331,31,40,0.775,60,224,284,107,25,31,52,70,360,582,2.06,0.48


In [44]:
data.shape

(8158, 30)

In [45]:
data.isna().sum()

Year           0
Season_type    0
PLAYER_ID      0
RANK           0
PLAYER         0
TEAM_ID        0
TEAM           0
GP             0
MIN            0
FGM            0
FGA            0
FG_PCT         0
FG3M           0
FG3A           0
FG3_PCT        0
FTM            0
FTA            0
FT_PCT         0
OREB           0
DREB           0
REB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
EFF            0
AST_TOV        0
STL_TOV        0
dtype: int64

In [46]:
data.drop(columns=['RANK','EFF'], inplace=True)

In [47]:
data['season_start_year'] = data['Year'].str[:4].astype(int)

In [48]:
data.TEAM.unique()

array(['OKC', 'NYK', 'MIA', 'MIN', 'LAC', 'GSW', 'HOU', 'TOR', 'IND',
       'DAL', 'POR', 'SAC', 'CHA', 'WAS', 'PHX', 'CLE', 'PHI', 'NOP',
       'BOS', 'MEM', 'ORL', 'ATL', 'MIL', 'DET', 'UTA', 'BKN', 'LAL',
       'SAS', 'DEN', 'CHI'], dtype=object)

In [49]:
data['Season_type'].replace('Regular%20Season','Regular_season', inplace = True)

In [50]:
rs_df = data[data['Season_type'] == 'Regular_season']
playoffs_df = data[data['Season_type'] == 'Playoffs']

In [51]:
data.columns

Index(['Year', 'Season_type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'AST_TOV', 'STL_TOV', 'season_start_year'],
      dtype='object')

In [52]:
total_cols = ['MIN' ,'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
             'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

In [57]:
# Group and calculate per-minute statistics
data_per_min = data.groupby(['PLAYER', 'PLAYER_ID', 'Year'])[total_cols].sum().reset_index()
for col in data_per_min.columns[4:]:
    data_per_min[col] = data_per_min[col] / data_per_min['MIN']

# Calculate additional metrics
data_per_min['FG%'] = data_per_min['FGM'] / data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M'] / data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM'] / data_per_min['FTA']
data_per_min['FG3A%'] = data_per_min['FG3A'] / data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS'] / data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M'] / data_per_min['FGM']
data_per_min['FTA/FGA%'] = data_per_min['FTA'] / data_per_min['FGA']
data_per_min['TRU%'] = 0.5 * data_per_min['PTS'] / (data_per_min['FGA'] + 0.475 * data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST'] / data_per_min['TOV']

# Filter out players with fewer than 50 minutes
data_per_min = data_per_min[data_per_min['MIN'] >= 50]

# Drop the PLAYER_ID column
data_per_min.drop(columns='PLAYER_ID', inplace=True)

# Select only numeric columns for correlation
numeric_cols = data_per_min.select_dtypes(include=['float64', 'int64'])

fig = px.imshow(numeric_cols.corr())
fig.show()
numeric_cols.corr()



,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,FG%,3PT%,FT%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA%,TRU%,AST_TOV
MIN,1.000000,0.441126,0.361457,0.193474,0.110037,0.349011,0.282716,-0.122380,0.048749,-0.020111,0.244821,0.032177,-0.054016,0.147085,-0.372827,0.468224,0.205142,0.185952,0.251545,-0.026143,0.273185,0.000364,0.037957,0.331403,0.057959
FGM,0.441126,1.000000,0.871810,0.190811,0.136154,0.626835,0.590241,0.078954,0.236240,0.190079,0.242314,-0.031584,0.071346,0.415833,-0.191788,0.958463,0.402646,0.121904,0.198854,-0.246520,0.360761,-0.235636,0.076741,0.417091,-0.102005
FGA,0.361457,0.871810,1.000000,0.424169,0.443907,0.565771,0.485546,-0.216465,-0.009909,-0.099175,0.344106,0.015630,-0.168800,0.436658,-0.320653,0.897965,-0.072520,0.170993,0.296083,0.036517,-0.022408,0.028629,-0.098849,0.031529,0.001484
FG3M,0.193474,0.190811,0.424169,1.000000,0.955652,-0.019753,-0.155668,-0.613035,-0.397525,-0.525304,0.149328,-0.012000,-0.433410,-0.068032,-0.406119,0.353581,-0.374718,0.578845,0.394713,0.832039,-0.086653,0.854055,-0.398798,0.118762,0.206111
FG3A,0.110037,0.136154,0.443907,0.955652,1.000000,-0.037000,-0.170822,-0.646632,-0.428884,-0.560426,0.164885,0.008827,-0.459750,-0.047780,-0.401490,0.298759,-0.512707,0.453667,0.372317,0.874750,-0.239937,0.840518,-0.427918,-0.063408,0.216503
FTM,0.349011,0.626835,0.565771,-0.019753,-0.037000,1.000000,0.955548,0.096775,0.207315,0.178567,0.253300,0.022138,0.074148,0.461538,-0.080574,0.749309,0.209791,0.009225,0.274897,-0.301703,0.487999,-0.294708,0.610281,0.283286,-0.115479
FTA,0.282716,0.590241,0.485546,-0.155668,-0.170822,0.955548,1.000000,0.240489,0.305725,0.305167,0.183303,0.021934,0.181459,0.465742,0.018950,0.679545,0.287932,-0.080216,0.033836,-0.419943,0.509710,-0.408051,0.719113,0.252658,-0.185228
OREB,-0.122380,0.078954,-0.216465,-0.613035,-0.646632,0.096775,0.240489,1.000000,0.669789,0.871006,-0.368513,-0.111713,0.603163,-0.003700,0.489697,-0.048444,0.565507,-0.406712,-0.364932,-0.650619,0.356143,-0.617895,0.398688,0.225036,-0.386218
DREB,0.048749,0.236240,-0.009909,-0.397525,-0.428884,0.207315,0.305725,0.669789,1.000000,0.948185,-0.215840,-0.128384,0.540095,0.129774,0.323448,0.146612,0.484343,-0.246808,-0.238275,-0.482457,0.350253,-0.462857,0.322034,0.259514,-0.327005
REB,-0.020111,0.190079,-0.099175,-0.525304,-0.560426,0.178567,0.305167,0.871006,0.948185,1.000000,-0.300477,-0.132738,0.615404,0.084275,0.423522,0.076271,0.562407,-0.335565,-0.313630,-0.597576,0.384112,-0.570607,0.383646,0.267981,-0.381168


Minutes PLayed Distibution

In [65]:
# Create the histogram
fig = px.histogram(x=rs_df['MIN'], nbins=30)  # You can adjust the number of bins as needed

# Update layout to label axes and add title
fig.update_layout(
    xaxis_title='Minutes Played',
    yaxis_title='Count of Players',
    title='Distribution of Minutes Played in Regular Season'
)

# Show the figure
fig.show()

In [63]:


# Create the figure
fig = go.Figure()

# Add the Regular Season histogram
fig.add_trace(go.Histogram(
    x=rs_df['MIN'] / rs_df['GP'], 
    histnorm='percent', 
    name='Regular Season',
    xbins={'start': 0, 'end': 46, 'size': 1}
))

# Add the Playoffs histogram
fig.add_trace(go.Histogram(
    x=playoffs_df['MIN'] / playoffs_df['GP'], 
    histnorm='percent', 
    name='Playoffs',
    xbins={'start': 0, 'end': 46, 'size': 1}
))

# Update the layout to overlay the histograms and set the opacity
fig.update_layout(
    barmode='overlay',
    xaxis_title='Minutes per Game',
    yaxis_title='Percentage of Players',
    title='Comparison of Minutes per Game: Regular Season vs Playoffs'
)
fig.update_traces(opacity=0.5)

# Show the figure
fig.show()
